### Εργασία: Επιχειρησιακή Έρευνα

**Ονοματεπώνυμο:** Παναγιώτης Κούτρης  
**ΑΕΜ:** 10671

In [101]:
import gurobipy as gp
from gurobipy import GRB

### Σχεδιασμός Εβδομαδιαίου Προγράμματος Μαθημάτων

Λύνουμε ένα πρόβλημα εβδομαδιαίου προγραμματισμού μαθημάτων για δύο σχολικές τάξεις, ώστε να ικανοποιούνται οι απαιτήσεις ωρών κάθε μαθήματος και οι περιορισμοί διαθεσιμότητας. 

Χρησιμοποιούμε γραμμικό ακέραιο προγραμματισμό με το Gurobi για να βρούμε ένα έγκυρο εβδομαδιαίο πρόγραμμα.

### Δημιουργία Μοντέλου

Ξεκινάμε δημιουργώντας το Gurobi μοντέλο.

In [52]:
model = gp.Model("WeeklyTimetable")

### Σύνολα

Ορίζουμε τα δύο τμήματα (class 1 και class 2) και τις 20 διαθέσιμες χρονικές ζώνες (5 ημέρες × 4 περίοδοι)

In [53]:
classes = [1, 2]
slots = list(range(20))

### Αντιστοίχιση Χρονικής Ζώνης σε Ημέρα και Περίοδο

Χρησιμοποιούμε δύο λεξικά:

- `slot_day[i]`: επιστρέφει σε ποια ημέρα (0=Δευτέρα, ..., 4=Παρασκευή) ανήκει το slot `i`
- `slot_period[i]`: επιστρέφει τη χρονική περίοδο της ημέρας (0=08:00–10:00, ..., 3=16:15–18:15)

Αυτό μας επιτρέπει να ομαδοποιούμε ή να περιορίζουμε μαθήματα με βάση τη μέρα και την ώρα

In [54]:
slot_day = {i: i // 4 for i in slots}
slot_period = {i: i % 4 for i in slots}

### Μαθήματα

Ορίζουμε τα μαθήματα με μοναδικό αριθμό και το όνομα του καθηγητή.

In [55]:
courses = {
    0: ("English", "Gesmanidis"),
    1: ("Biology", "Insoulina"),
    2: ("History-Geography", "Hartoula"),
    3: ("Math_class1", "Antiparagwgos"),
    4: ("Math_class2", "Lathopraxis"),
    5: ("Physics", "Kirkofidou"),
    6: ("Philosophy", "Platizwn"),
    7: ("PE_class1", "Bratsakis"),
    8: ("PE_class2", "Trexalitoula")
}

### Μεταβλητές Απόφασης

Ορίζουμε δυαδικές μεταβλητές $D[s, c, t]$ που είναι 1 αν το μάθημα (course) $c$ διδάσκεται στην τάξη $s$ στη χρονική στιγμή $t$

In [56]:
D = model.addVars(
    classes,
    courses.keys(),
    slots,
    vtype=GRB.BINARY,
    name="D"
)

### Απαιτούμενες Ώρες Ανά Μάθημα

Ορίζουμε πόσες φορές (δίωρα) πρέπει να διδάσκεται κάθε μάθημα ανά τάξη κατά τη διάρκεια της εβδομάδας

In [57]:
# (class, course): 2_hour_sessions
required_hours = {
    (1, 0): 1,
    (2, 0): 1,
    (1, 1): 3,
    (2, 1): 3,
    (1, 2): 2,
    (2, 2): 2,
    (1, 3): 4,
    (2, 4): 4,
    (1, 5): 3,
    (2, 5): 3,
    (1, 6): 1,
    (2, 6): 1,
    (1, 7): 1,
    (2, 8): 1
}

### Περιορισμός: Ένα Μάθημα Ανά Χρονική Ζώνη

Κάθε class μπορεί να έχει το πολύ ένα μάθημα σε κάθε χρονική ζώνη (slot).


In [58]:
for s in classes:
    for t in slots:
        model.addConstr(gp.quicksum(D[s, c, t] for c in courses.keys()) <= 1,
                        name=f"OneCoursePerSlot_class{s}_slot{t}")

### Περιορισμός: Συνολικές Ώρες

Κάθε μάθημα πρέπει να διδάσκεται ακριβώς όσες φορές απαιτείται για κάθε class.


In [59]:
for (s, c), hours in required_hours.items():
    model.addConstr(gp.quicksum(D[s, c, t] for t in slots) == hours, name=f"Required_class{s}_course{c}")

### Περιορισμός: Όχι Ίδιο Μάθημα Πάνω από Μία Φορά Την Ίδια Ημέρα

Για κάθε μάθημα και κάθε class, δεν επιτρέπεται να προγραμματιστεί πάνω από μία φορά την ίδια ημέρα. Αυτό αποτρέπει την επανάληψη του ίδιου μαθήματος σε συνεχόμενες ή διαφορετικές ζώνες της ίδιας ημέρας.

In [60]:
for s in classes:
    for c in courses.keys():
        for day in range(5):  # Days 0 to 4
            daily_slots = [t for t in slots if slot_day[t] == day]
            model.addConstr(gp.quicksum(D[s, c, t] for t in daily_slots) <= 1,
                            name=f"NoRepeat_class{s}_course{c}_day{day}")

### Περιορισμοί Διαθεσιμότητας

Ορίζουμε χρονικές ζώνες που δεν επιτρέπεται να προγραμματιστεί κάποιο μάθημα λόγω απουσίας καθηγητών ή άλλων περιορισμών:

- Η πρώτη ζώνη της Δευτέρας (slot 0) είναι δεσμευμένη
- Η κα Ινσουλίνα (Biology) δεν διδάσκει Τετάρτη (day 2)
- Ο κος Λαθοπράξης (Math για την class 2) δεν διδάσκει Δευτέρα πρωί (slot 0 και slot 1)

In [61]:
for s in classes:
    for c in courses.keys():
        model.addConstr(D[s, c, 0] == 0, name=f"NoMondayMorning_class{s}_course{c}")

biology_id = 1
for s in classes:
    for t in slots:
        if slot_day[t] == 2:
            model.addConstr(D[s, biology_id, t] == 0, name=f"NoInsoulinaWed_class{s}_slot{t}")

math_class2_id = 4
for t in [0, 1]:  
    model.addConstr(D[2, math_class2_id, t] == 0, name=f"NoLathopraxisMonMorning_slot{t}")


### Περιορισμός: Φυσική Αγωγή Μόνο Πέμπτη 14:00–16:00

Η Φυσική Αγωγή πρέπει να διδάσκεται μόνο την Πέμπτη, στη ζώνη 14:00–16:00 (slot 14). Σε όλα τα υπόλοιπα slots απαγορεύεται.


In [62]:
model.addConstr(D[1, 7, 14] == 1, name="PE_class1_at_slot14")
model.addConstr(D[2, 8, 14] == 1, name="PE_class2_at_slot14")

for t in slots:
    if t != 14:
        model.addConstr(D[1, 7, t] == 0, name=f"NoPE_class1_slot{t}")
        model.addConstr(D[2, 8, t] == 0, name=f"NoPE_class2_slot{t}")

### Περιορισμός: Κοινός Καθηγητής Δεν Διδάσκει Ταυτόχρονα Και Στις Δύο Classes

Για κάθε slot, αν ένας καθηγητής διδάσκει και στις δύο classes, δεν επιτρέπεται να έχει μάθημα σε αυτές ταυτόχρονα.

In [63]:
for t in slots:
    for teacher in set(teacher for _, teacher in courses.values()):
        course_ids = [c for c, (_, prof) in courses.items() if prof == teacher]
        model.addConstr(
            gp.quicksum(D[s, c, t] for s in classes for c in course_ids if (s, c) in required_hours) <= 1,
            name=f"NoTeacherOverlap_{teacher}_slot{t}"
        )

### Επίλυση Μοντέλου και Εμφάνιση Αποτελέσματος

Κάνουμε optimize το μοντέλο. Αν βρεθεί εφικτή λύση, εμφανίζουμε το πρόγραμμα για κάθε class.

In [64]:
model.optimize()

if model.Status == GRB.OPTIMAL or model.Status == GRB.FEASIBLE:
    for s in classes:
        print(f"\n--- Timetable for class {s} ---")
        for t in slots:
            for c in courses.keys():
                if D[s, c, t].X > 0.5:
                    day = ["Mon", "Tue", "Wed", "Thu", "Fri"][slot_day[t]]
                    period = slot_period[t]
                    course_name = courses[c][0]
                    print(f"{day} period {period}: {course_name}")
else:
    print("No feasible solution found.")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 392 rows, 360 columns and 1348 nonzeros
Model fingerprint: 0xcdd5f19b
Variable types: 0 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

--- Timetable for class 1 ---
Mon period 1: Physics
Mon period 2: Math_class1
Mon period 3: Biology
Tue period 0: History-Geography
Tu

### Τελική Εμφάνιση Προγράμματος σε Πίνακα

Χρησιμοποιούμε pandas για να εμφανίσουμε καθαρά το εβδομαδιαίο πρόγραμμα κάθε class.

In [65]:
import pandas as pd

days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
periods = ["08:00–10:00", "10:15–12:15", "14:00–16:00", "16:15–18:15"]

for s in classes:
    table = pd.DataFrame(index=periods, columns=days)
    for t in slots:
        for c in courses.keys():
            if D[s, c, t].X > 0.5:
                day = days[slot_day[t]]
                period = periods[slot_period[t]]
                course_name = courses[c][0]
                table.at[period, day] = course_name
    print(f"\n--- Timetable for class {s} ---")
    display(table.fillna("-"))


--- Timetable for class 1 ---


,Mon,Tue,Wed,Thu,Fri
08:00–10:00,-,History-Geography,Physics,Biology,Math_class1
10:15–12:15,Physics,Math_class1,Philosophy,History-Geography,-
14:00–16:00,Math_class1,Physics,-,PE_class1,English
16:15–18:15,Biology,-,-,Math_class1,Biology



--- Timetable for class 2 ---


,Mon,Tue,Wed,Thu,Fri
08:00–10:00,-,-,Philosophy,-,Biology
10:15–12:15,History-Geography,-,-,Math_class2,Math_class2
14:00–16:00,Biology,Biology,Math_class2,PE_class2,Physics
16:15–18:15,Math_class2,Physics,Physics,English,History-Geography


### Βέλτιστη Χωροθέτηση Αποθηκών

Ελαχιστοποιούμε το συνολικό κόστος που αποτελείται από:

- Το κόστος ανοίγματος των αποθηκών που επιλέγονται
- Το κόστος μεταφοράς των προϊόντων από τις αποθήκες στους πελάτες

Χρησιμοποιούμε μεικτό γραμμικό προγραμματισμό με το Gurobi για την εύρεση της βέλτιστης λύσης.

### Δημιουργία Νέου Μοντέλου

Ορίζουμε Gurobi μοντέλο 

In [66]:
model2 = gp.Model("WarehouseLocation")

### Ορισμός Συνόλων

Ορίζουμε τα βασικά σύνολα του προβλήματος:

- 12 αποθήκες: αριθμημένες από 1 έως 12
- 12 κέντρα πώλησης: αριθμημένα επίσης από 1 έως 12

In [67]:
warehouses = list(range(1, 13))
centers = list(range(1, 13))

### Ορισμός Παραμέτρων

Καταχωρούμε:

- Κόστος ανοίγματος και χωρητικότητα για κάθε αποθήκη (Πίνακας 3)
- Ζήτηση κάθε κέντρου πώλησης (Πίνακας 4)

In [68]:
fixed_cost = {
    1: 3500, 2: 9000, 3: 10000, 4: 4000, 5: 3000, 6: 9000,
    7: 9000, 8: 3000, 9: 4000, 10: 10000, 11: 9000, 12: 3500
}

capacity = {
    1: 300, 2: 250, 3: 100, 4: 180, 5: 275, 6: 300,
    7: 200, 8: 220, 9: 270, 10: 250, 11: 230, 12: 180
}

demand = {
    1: 120, 2: 80, 3: 75, 4: 100, 5: 110, 6: 100,
    7: 90, 8: 60, 9: 30, 10: 150, 11: 95, 12: 120
}

### Κόστος Μεταφοράς ανά Τόνο

Ορίζουμε τον πίνακα με τα συνολικά κόστη μεταφοράς από κάθε αποθήκη σε κάθε κέντρο πώλησης, όπως δίνεται στην εκφώνηση.  
Με βάση τη ζήτηση κάθε κέντρου, υπολογίζουμε το κόστος ανά τόνο με απλή διαίρεση.

In [69]:
full_cost_matrix = [
    [100, 80, 50, 50, 60, 120, 90, 60, 70, 65, 110, 100],
    [120, 90, 60, 70, 65, 110, 140, 110, 80, 60, 75, 130],
    [100, 80, 70, 100, 75, 130, 130, 90, 80, 120, 80, 150],
    [160, 125, 100, 90, 100, None, None, 130, 60, 75, 65, 100],
    [190, 150, None, None, None, 50, 50, 45, 60, 100, 100, 100],
    [200, 180, 150, None, None, 90, 90, 50, 60, 75, 100, 90],
    [200, 180, 150, None, None, 80, 60, 50, 70, 60, 70, 90],
    [120, 90, 70, None, None, 70, 60, 60, 60, 100, 80, 100],
    [140, 110, 90, None, None, 100, 125, 100, 80, 75, 100, 90],
    [160, 125, 100, None, None, None, None, 150, None, 100, 80, None],
    [200, 180, 150, None, None, None, None, None, 100, 50, 60, 90],
    [200, 180, 150, None, None, None, None, None, None, 60, 80, 100]
]

transport_cost = {}

for w in range(12):
    for c in range(12):
        cost = full_cost_matrix[w][c]
        if cost is not None:
            transport_cost[(w + 1, c + 1)] = round(cost / demand[c + 1], 5)


### Μεταβλητές Απόφασης

Ορίζουμε δύο τύπους μεταβλητών:

- `y[j]`: είναι 1 αν ανοίξει η αποθήκη j, διαφορετικά 0
- `x[j, i]`: ποσότητα που αποστέλλεται από την αποθήκη j στο κέντρο i

In [70]:
y = model2.addVars(warehouses, vtype=GRB.BINARY, name="Open")
x = model2.addVars(warehouses, centers, vtype=GRB.CONTINUOUS, name="Flow")

### Αντικειμενικός Σκοπός

Ελαχιστοποίηση κόστους

In [71]:
model2.setObjective(
    gp.quicksum(fixed_cost[j] * y[j] for j in warehouses) +
    gp.quicksum(transport_cost[j, i] * x[j, i]
                for j, i in transport_cost.keys()),
    GRB.MINIMIZE
)

### Περιορισμός: Κάλυψη Ζήτησης

Κάθε κέντρο πώλησης πρέπει να λάβει ακριβώς τη ζητούμενη ποσότητα από τις αποθήκες.

In [72]:
for i in centers:
    model2.addConstr(
        gp.quicksum(x[j, i] for j in warehouses if (j, i) in transport_cost) == demand[i],
        name=f"Demand_{i}"
    )

### Περιορισμός: Χωρητικότητα Αποθήκης

Η συνολική ποσότητα που αποστέλλει κάθε αποθήκη δεν επιτρέπεται να ξεπερνά τη χωρητικότητά της.  
Ο περιορισμός ενεργοποιείται μόνο αν η αποθήκη έχει επιλεγεί (δηλαδή y[j] = 1).

In [73]:
for j in warehouses:
    model2.addConstr(
        gp.quicksum(x[j, i] for i in centers if (j, i) in transport_cost) <= capacity[j] * y[j],
        name=f"Capacity_{j}"
    )

### Επίλυση και Εμφάνιση Αποτελεσμάτων

Κάνουμε optimize το μοντέλο και εμφανίζουμε:

- Ποιες αποθήκες θα ανοίξουν
- Τι ποσότητα στέλνει κάθε αποθήκη σε κάθε κέντρο πώλησης

In [74]:
model2.optimize()

if model2.Status == GRB.OPTIMAL or model2.Status == GRB.FEASIBLE:
    print("Open Warehouses:")
    for j in warehouses:
        if y[j].X > 0.5:
            print(f"  - Warehouse {j}")

    print("\nShipments:")
    for j, i in transport_cost:
        quantity = x[j, i].X
        if quantity > 1e-6:
            print(f"  From warehouse {j} to center {i}: {quantity} tons")
else:
    print("No feasible solution found.")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 156 columns and 240 nonzeros
Model fingerprint: 0x6d64740e
Variable types: 144 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective 69173.334150
Presolve removed 0 rows and 30 columns
Presolve time: 0.00s
Presolved: 24 rows, 126 columns, 240 nonzeros
Variable types: 114 continuous, 12 integer (12 binary)

Root relaxation: objective 1.558139e+04, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

### Εμφάνιση Αποτελεσμάτων σε Πίνακα

Παρουσιάζουμε τις αποστολές από τις αποθήκες προς τα κέντρα πώλησης σε μορφή πίνακα για πιο εύκολη ανάγνωση.

In [75]:
import pandas as pd

shipment_table = pd.DataFrame(index=centers, columns=warehouses)

for j, i in transport_cost:
    shipment_table.at[i, j] = round(x[j, i].X, 2)

shipment_table = shipment_table.infer_objects(copy=False)
shipment_table = shipment_table.fillna(0).astype(float)

shipment_table.index.name = "Center"
shipment_table.columns.name = "Warehouse"
display(shipment_table)

open_warehouses = [j for j in warehouses if y[j].X > 0.5]
print("\nOpen Warehouses:", open_warehouses)

Warehouse,1,2,3,4,5,6,7,8,9,10,11,12
Center,,,,,,,,,,,,
1,15.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0
3,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0



Open Warehouses: [1, 4, 5, 8, 12]
